## Strategy Idea 2 : "Cointegration - pairs trading"

__Section 0: Setup__ Importing packages/reading in data etc.

__Section 1 : Idea__ 

- __1.1__ Strategy idea

- __1.2__ Origin of idea. Context/Reasoning for strategy to work e.g. use in financial markets?

__Section 2 : Exploration__

- __2.1__ Exploratory Data Analysis. e.g plots of price/volumes that could show strategy working, how much potential.

- __2.2__ Define some 'strategy metrics'. Metrics that can can you use to gauge if this strategy will work i.e no.price points above a certain threshold that is profitable. Metrics could show how often there is an opportunity to make a trade and how much 'value' is in an opportunity e.g. how much is there a price swing?


__Section 3 : Strategy testing__

- __3.1__ Testing strategy on previous data. 

- __3.2__ State any assumptions made by testing.

- __3.3__ Model refinements. How could strategy be optimised? Careful : is this backfitting/overfitting - what measures taken to negate this e.g. bootstrapping?

- __3.4__ Assessing strategy. P/L on data sample? ROI? variance in results? longest losing run?

__Section 4 : Practical requirements__

- __4.1__ Identify if this edge is ‘realisable’? What methods will you apply to extract this value? e.g. applying a hedge function


- __4.2__ Is it possible to quantify the potential profit from the strategy? Consideration : How long will it take to obtain this? How 'risky' is it? e.g. if something did go wrong, how much do we lose? 

- __4.3__ Strategy limitations. The factors that could prevent strategy working e.g. practical considerations e.g. reacting quick enough to market updates, volume behind a price, size of bankroll needed


__Section 5: Potential limitations__

- __5.1__ What is our 'competition' - if not quantifiable, do we suspect people are doing the same thing? 

- __5.2__ So what's our edge? Identify ways of finding this edge in future? e.g what features are there? Are they predictive? Is there a certain 'market/runner' profile?





Notes (to do)
* Use lay prices as well (currently only using back prices, but two lay bets are made per pairs trade

### Section 0 : Setup

In [35]:
# importing packages
from pathlib import Path, PurePath 

import pandas as pd
import numpy as np
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

import itertools

import utils

In [36]:
# reading in data
project_dir = Path.cwd().parents[2]
data_dir = project_dir / 'data' / 'processed' / 'api' / 'advanced' / 'adv_data.csv'
df = pd.read_csv(data_dir, index_col = 0)
print(df.shape)

# defining variables
back_prices = [col for col in df.columns if 'BP' in col]
back_sizes = [col for col in df.columns if 'BS' in col]
lay_prices = [col for col in df.columns if 'LP' in col]
lay_sizes = [col for col in df.columns if 'LS' in col]

df.head()

(13073, 307)


,SelectionId,MarketId,Venue,Distance,RaceType,BSP,NoRunners,BS:T-60,BS:T-59,BS:T-58,...,LS:T+5,LS:T+6,LS:T+7,LS:T+8,LS:T+9,LS:T+10,LS:T+11,LS:T+12,LS:T+13,LS:T+14
0,11986132,1.169028,Huntingdon,20.0,Chase,8.33,9,16.43,24.51,26.57,...,10.08,11.15,5.44,7.09,14.16,19.53,3.12,3.31,0.68,0.68
1,16800725,1.169028,Huntingdon,20.0,Chase,3.68,9,15.43,25.74,57.82,...,29.87,221.22,43.23,43.10,13.53,26.15,13.60,74.30,419.52,23082.10
2,20968322,1.169028,Huntingdon,20.0,Chase,14.96,9,9.87,9.25,9.15,...,37.32,6.83,4.85,11.23,16.00,5.68,40.25,12.51,10.42,13.17
3,22023486,1.169028,Huntingdon,20.0,Chase,4.25,9,84.38,64.49,58.01,...,11.67,2.02,2.02,2.02,2.02,2.02,2.02,2.02,2.02,2.02
4,24496216,1.169028,Huntingdon,20.0,Chase,6.60,9,10.64,10.11,7.91,...,34.27,54.72,11.85,17.99,48.21,17.28,38.29,6.96,4.37,4.37


### Section 1 : Idea

__1.1 Idea__

Prices in a market adjust such that the sum of the implied odds of all horses is approximately equal to one. If one horse's price drifts, another's or several other's should be backed in. 

One traditional strategy considering this price behaviour is 'pairs trading': *"A pairs trade or pair trading is a market neutral trading strategy enabling traders to profit from virtually any market conditions: uptrend, downtrend, or sideways movement. This strategy is categorized as a statistical arbitrage and convergence trading strategy."*

__1.2  Reasoning__

Why is there an edge here?
- .

### Section 2 : Pairs trading

__2.1__ **- Idea**

[Bebbington, PA (2017)](https://discovery.ucl.ac.uk/id/eprint/1563501/) looks at pairs trading in horse racing markets. The following outlines their method for analysing this strategy. In 2.2, each step will be attempted.

* The 'signals' are the best match or lay price available at a given timestamp.
* Statsitical methods are used to analyse horses' pricing data for comparison, in particular to calcuate a hedge ratio and for stake weighting. In the paper, non-overlapping windows of data, for example, price observations 1-5, 6-10, 11-15, make up the time series, and then trades are made at the end of the window. This is used to simulate a method where the algorithm is reacting to live data. This example study movement throughout 30 price points and make bets in the remaining 30 periods.
* A z-score transformation of the log of the decimal odds is used to standardise prices. This makes the relative directional movement in different prices comparable by accounting for their respective variances. 
* Pairs are discovered by analysing the sum of squared distances between two horses' prices throughout time. Those that move the least relative to each other are the best candidates for pairs.
* Once pairs are identified, the 'spread' between their prices (on average, or at the end of each window) is compared to a minimum size requirement for a bet to be made, $\phi$.
* The 'hedging ratio' is found using an OLS regression of the price of one of the horses on the price of the other. Since the two prices are pairs but will have different variances and absolute values, their movement relative to eachother must be considered to make the strategy 'cost neutral'. It is also used to define the stake size. 
* The final observed spread indicates which on which horse a 'back-to-lay' hedge must be made (that which is expected to be backed in) and on which a 'lay-to-back' hedge must be made (that which is expected to drift). This spread is compared to an interval [?], likely a confidence interval of past spreads or simply the interval of observed spreads. If the spread is greater than usual or smaller than usual, the bets are placed. 
* In the paper it appears that both sides of the hedge bet are made at the same point in time.

__2.2__ **- Setup**

**Data**

The following example will be set up with a random race and will identify tradeable pairs (or that there are none). Three DataFrames are created: (1) the unchanged race sample DataFrame with one row per horse and data going along in columns, (2) a back prices DataFrame with one column per horse and prices going through time in rows, (3) the same for lay prices. This analysis looks at prices before the race begins.

There are 60 price data points for each horse, finishing at the begining of the race.

Variables:
* $BP_{t}^{i}$ is back price for horse i at time t.
* $LP_{t}^{i}$ is lay price for horse i at time t.

In [37]:
#runner_info = ['SelectionId', 'MarketId', 'Venue', 'Distance', 'RaceType', 'BSP', 'NoRunners']

sample_df = df[df['MarketId'] == df['MarketId'].sample(1).item()]

bp_df = sample_df[['SelectionId'] + back_prices].copy()
new_cols = bp_df.columns.str.replace("[BP:T]", "").str.replace("[+]", "")
bp_df.rename(columns = dict(zip(bp_df.columns, new_cols)), inplace = True)
bp_t_df = bp_df.T.copy()
bp_t_df.columns = ["h" + str(int(column)) for column in bp_t_df.iloc[0]]
bp_t_df = bp_t_df.iloc[1:-15] # using the 60 pre-off price data points
bp_t_df.reset_index(drop=True, inplace=True)

lp_df = sample_df[['SelectionId'] + lay_prices].copy()
new_cols = lp_df.columns.str.replace("[LP:T]", "").str.replace("[+]", "")
lp_df.rename(columns = dict(zip(lp_df.columns, new_cols)), inplace = True)
lp_t_df = lp_df.T.copy()
lp_t_df.columns = ["h" + str(int(column)) for column in lp_t_df.iloc[0]]
lp_t_df = lp_t_df.iloc[1:-15]
lp_t_df.reset_index(drop=True, inplace=True)

# bsp_df = plot_df[['BSP']].copy()
# bsp_df['min_bp'] = bsp_df['BSP'].apply(lambda x: round(utils.back_hedge_min_bp(x, 0.05), 2))
# bsp_df['max_lp'] = bsp_df['BSP'].apply(lambda x: round(utils.lay_hedge_max_lp(x, 0.05), 2))    

bp_t_df.head()

,h10460778,h12464466,h13168800,h15609001,h15888970,h21809948,h22972495,h26684163,h3518738,h5826525,h6406084,h6670738
0,46.0,11.00,28.00,9.32,4.30,4.4,6.0,30.00,36.00,100.0,19.00,55.0
1,46.0,10.96,28.19,9.32,4.33,4.4,6.0,30.00,37.72,100.0,19.48,55.0
2,46.0,10.50,29.00,9.20,4.29,4.4,6.0,30.26,36.18,100.0,19.50,55.0
3,46.0,10.50,29.85,9.20,4.30,4.4,6.0,30.00,38.00,100.0,19.67,55.0
4,46.0,10.70,30.00,9.20,4.29,4.4,6.0,30.00,38.64,100.0,19.75,55.0


__2.3__ **- Z-score transformation**

Bebbington standardises prices by taking the natural logarithm of each price and then standardising it with a Z-score transformation. 

Taking $P_{t}^{i} = ln(BP_{t}^{i})$, this means finding 

#### $P_{t}^{'(i)} = \frac{P_{t}^{i}-\overline{P}_{t}^{i}}{\sigma^{(i)}}$

where $\sigma^{(i)}$ is the standard deviation of the horse's price throughout the time series.

The Z-score transformation gives the relationship between an individual data point in the sample relative to that of the population mean and standard deviation. This means that variations are comparable between horses.

The following will look at the first 30 observations.

In [38]:
z_bp_df = bp_t_df.copy()
z_bp_df = z_bp_df[:30] #first 30 observations
z_bp_df = np.log(z_bp_df)

for column in z_bp_df.columns:
    mean = z_bp_df[column].mean()
    sd = np.std(z_bp_df[column], ddof = 1)
    z_bp_df[column] = z_bp_df[column].apply(lambda x: (x - mean) / sd)
    
z_bp_df.head()

,h10460778,h12464466,h13168800,h15609001,h15888970,h21809948,h22972495,h26684163,h3518738,h5826525,h6406084,h6670738
0,-1.209473,-0.568406,-2.315551,1.526762,0.448808,1.799521,1.230401,-1.030462,-1.319054,-0.190896,-0.213962,-0.357427
1,-1.209473,-0.659024,-2.175114,1.526762,0.733407,1.799521,1.230401,-1.030462,-0.244004,-0.190896,0.501174,-0.357427
2,-1.209473,-1.725579,-1.586840,1.061899,0.353500,1.799521,1.230401,-0.964068,-1.204169,-0.190896,0.530587,-0.357427
3,-1.209473,-1.725579,-0.986926,1.061899,0.448808,1.799521,1.230401,-1.030462,-0.073648,-0.190896,0.779392,-0.357427
4,-1.209473,-1.256230,-0.882835,1.061899,0.353500,1.799521,1.230401,-1.030462,0.311069,-0.190896,0.895733,-0.357427


__2.4__ **- Sum of squared distances**

Following Bebbington, to select pairs, we create a matrix (DataFrame, in this case) of the sum of squared distances between pairs of horses throughout the time series.

$\Theta _{ij} = \left\{\begin{matrix}
\sum_{M}^{t=1}(P_{t}^{'(i)} - P_{t}^{'(j)})^{2}, &i\neq j\\ 
 0, i=j& 
\end{matrix}\right.$

In [39]:
ids = [column for column in z_bp_df.columns]

matrix = z_bp_df.iloc[0:0].copy()

matrix.insert(0, "horse", np.array(ids))
matrix = matrix.set_index("horse", drop = False)
del matrix["horse"]

for column in matrix.columns:
    for row in matrix.index:
        if column == row:
            matrix.loc[row, column] = np.nan
        else: 
            matrix.loc[row, column] = ((z_bp_df[row] - z_bp_df[column]) ** 2).sum() or np.nan
            
for x in range(len(ids)):
    for y in range(x, len(ids)):
        matrix.iloc[x, y] = np.nan
        
matrix

,h10460778,h12464466,h13168800,h15609001,h15888970,h21809948,h22972495,h26684163,h3518738,h5826525,h6406084,h6670738
horse,,,,,,,,,,,,
h10460778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h12464466,19.442460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h13168800,19.533678,17.911402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h15609001,107.425631,98.610129,97.501976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h15888970,88.688792,99.567096,86.387575,26.417339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h21809948,93.802219,90.341672,92.698417,30.741600,44.392414,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h22972495,85.268878,67.182760,86.072764,38.052781,71.596658,33.123379,NaN,NaN,NaN,NaN,NaN,NaN
h26684163,9.937667,10.563164,19.203957,104.160797,106.141735,86.487667,62.060403,NaN,NaN,NaN,NaN,NaN
h3518738,23.937538,32.896941,29.683790,90.596988,102.270741,68.219612,50.888431,14.798555,NaN,NaN,NaN,NaN


In [40]:
horse_x = matrix.min(axis=1).idxmin()
horse_y = matrix.min().idxmin()
sss = matrix.min().min()

print(f"Pair found: horse {horse_x} and horse {horse_y} with sum of squared spreads equal to {sss}.")

Pair found: horse h26684163 and horse h10460778 with sum of squared spreads equal to 9.937667046775577.


__2.5__ **- Regression of prices of horse Y on horse X**

With $x_{t} = \left \{P_{1}^{X} + P_{2}^{X} + , ... , P_{M}^{X} \right \}$ and  $y_{t} = \left \{P_{1}^{Y} + P_{2}^{Y} + , ... , P_{M}^{Y} \right \}$ where $M$ is the final time period in the window, we carry out the OLS regression of $y_{t}$ on $x_{t}$. The estimate of $\beta$ is the hedging ratio, giving the relative holding of $x_{t}$ for a cost-neutral hedge position.

$y_{t} = \beta x_{t} + \varepsilon_{t}$

Using this estimation and the final end of window observations at time $M$ we get the spread at the end of the window.

$\varepsilon_{M} = y_{M} - \hat{\beta} x_{M}$.

If $\varepsilon_{M}$ is outside of an interval of past spread values such that if the spread returns to the mean a hedge bet will be profitable, bets can be made.

In [41]:
#regression setup
reg_df = bp_t_df[[horse_y, horse_x]][:30].copy() #non-standardised prices
reg_df = np.log(reg_df) 
reg_df['const'] = 1

In [42]:
#regression fit and results
reg = sm.OLS(endog=reg_df[horse_y], exog=reg_df[['const', horse_x]], missing='drop')

results = reg.fit()

#print(results.summary())

constant = results.params[0]
beta = results.params[1]
print(f"\nHedge ratio beta = {beta}.")


Hedge ratio beta = 1.1238714640116556.


In [43]:
#estimated final period (T=30) spread
spread = reg_df[horse_y].iloc[29].item() - constant - beta * reg_df[horse_x].iloc[29].item()

print(f"Final period estimated spread (in log prices) epsilon = {spread}.")

if spread > 0:
    print("Positive spread: horse Y has drifted from the mean, horse X has been backed in. If mean reversion occurs horse Y will be backed in and horse X will drift. Back-to-lay hedge Y and lay-to-back hedge X.")
else:
    print("Negative spread: horse X has drifted from the mean, horse Y has been backed in. If mean reversion occurs horse X will be backed in and horse Y will drift. Back-to-lay hedge X and lay-to-back hedge Y.")

Final period estimated spread (in log prices) epsilon = -0.22514353753674765.
Negative spread: horse X has drifted from the mean, horse Y has been backed in. If mean reversion occurs horse X will be backed in and horse Y will drift. Back-to-lay hedge X and lay-to-back hedge Y.


__2.6__ **- Estimated residual vs. 'threshold'**

[Return to this later]

__2.7.1__ **- Pairs trade**

Bebbing describes the following steps for the pairs trade:

If the final estimated residual $\varepsilon_{M}$ is:
* Less than the lower bound of the threshold: back horse Y with £1, lay horse X with £$\beta$.
* Greater than the upper bound of the threshold: lay horse Y with £1 and back horse X with £$\beta$.

Bebbington describes using a self-imposed delay of 5 seconds to emulate the delays in placing a bet on the Betfair Exchange. Since each time step in this dataset before the beginning of the race is 2 minutes, 1 step will be used. The prices at time 31 (index = 30) will be used.

This is followed in the cell below. Put alone, this strategy doesnt make sense. The bettor is still exposed to the result of the race without closing out each position with an opposing bet on each horse after some period of time.

In [44]:
# #a threhold of zero is going to be used initially

# def payout(bp, bs, lp, ls, c = 0):
#     loss_side = - bs + ls * (1 - c) 
#     win_side = (bp - 1) * bs * (1 - c) - (lp - 1) * ls
#     return win_side, loss_side

# if spread < 0:
#     bp = bp_t_df[horse_y].iloc[30]
#     lp = bp_t_df[horse_x].iloc[30]
#     win_side, loss_side = payout(bp, 1, lp, beta)
#     print(f"Win side = {win_side}. Loss side = {loss_side}.")
# else:
#     bp = bp_t_df[horse_x].iloc[30]
#     lp = bp_t_df[horse_y].iloc[30]
#     win_side, loss_side = payout(bp, beta, lp, 1)
#     print(f"Win side = {win_side}. Loss side = {loss_side}.")

__2.7.2__ **- Pairs trade 2**

In a pairs trade with two financial assets (say, company shares) the objective is to go long on the share whose price is expected to increase, while shorting the share whose price is expected to decrease. In both cases the positions would be closed out: the asset is sold or bought back (in the case of the short-sale) once there is a gain or possibly after a set interval. In the prior example, the closing out of each position should be done via an opposite back/lay.

If the final estimated residual $\varepsilon_{M}$ is:
* Less than the lower bound of the threshold: back horse Y with £1, lay horse X with £$\beta$. **After k periods, lay horse Y and back horse X with the optimal stakes given the prevailing prices, as defined in utils.py.**
* Greater than the upper bound of the threshold: lay horse Y with £1 and back horse X with £$\beta$. **After k periods, back horse Y and lay horse X with the optimal stakes.**

In [45]:
# k = 35

# if spread < 0:
#     #back to lay Y
#     bp_y = bp_t_df[horse_y].iloc[30]
#     lp_y = bp_t_df[horse_y].iloc[k]
    
#     win_side_y, loss_side_y = utils.payout(bp_y, 1, lp_y, '?')
#     print(f"Win side = {win_side_y}. Loss side = {loss_side_y}.")
    
#     #lay to back X
#     lp_x = bp_t_df[horse_x].iloc[30]
#     bp_x = bp_t_df[horse_x].iloc[k]
    
#     win_side_x, loss_side_x = utils.payout(bp_x, '?', lp_x, beta)
#     print(f"Win side = {win_side_x}. Loss side = {loss_side_x}.")
        
# else:
#     #lay to back Y
#     lp_y = bp_t_df[horse_y].iloc[30]
#     bp_y = bp_t_df[horse_y].iloc[k]
    
#     win_side_y, loss_side_y = utils.payout(bp_y, '?', lp_y, 1)
#     print(f"Win side = {win_side_y}. Loss side = {loss_side_y}.")
    
#     #back to lay X
#     bp_x = bp_t_df[horse_x].iloc[30]
#     lp_x = bp_t_df[horse_x].iloc[k]
    
#     win_side_x, loss_side_x = utils.payout(bp_x, beta, lp_x, '?')
#     print(f"Win side = {win_side_x}. Loss side = {loss_side_x}.")

__2.9__ **- Monte Carlo simulation**

Below is a Monte Carlo simulation of full process outlined above, repeated 1,000 times with profits aggregated.

In [46]:
# def payout(bp, bs, lp, ls, c = 0):
#     loss_side = - bs + ls * (1 - c) 
#     win_side = (bp - 1) * bs * (1 - c) - (lp - 1) * ls
#     return win_side, loss_side

# returns = 0

# for n in range(1000):
#     sample_df = df[df['MarketId'] == df['MarketId'].sample(1).item()].copy()

#     bp_df = sample_df[['SelectionId'] + back_prices].copy()
#     new_cols = bp_df.columns.str.replace("[BP:T]", "").str.replace("[+]", "")
#     bp_df.rename(columns = dict(zip(bp_df.columns, new_cols)), inplace = True)
#     bp_t_df = bp_df.T.copy()
#     bp_t_df.columns = ["h" + str(column) for column in bp_t_df.iloc[0]]
#     bp_t_df = bp_t_df.iloc[1:-15] # using the 60 pre-off price data points
#     #bp_t_df.astype(int)
#     bp_t_df.reset_index(drop=True, inplace=True)

#     lp_df = sample_df[['SelectionId'] + lay_prices].copy()
#     new_cols = lp_df.columns.str.replace("[LP:T]", "").str.replace("[+]", "")
#     lp_df.rename(columns = dict(zip(lp_df.columns, new_cols)), inplace = True)
#     lp_t_df = lp_df.T.copy()
#     lp_t_df.columns = ["h" + str(column) for column in lp_t_df.iloc[0]]
#     lp_t_df = lp_t_df.iloc[1:-15]
#     lp_t_df.reset_index(drop=True, inplace=True)
    
#     # bsp_df = plot_df[['BSP']].copy()
#     # bsp_df['min_bp'] = bsp_df['BSP'].apply(lambda x: round(utils.back_hedge_min_bp(x, 0.05), 2))
#     # bsp_df['max_lp'] = bsp_df['BSP'].apply(lambda x: round(utils.lay_hedge_max_lp(x, 0.05), 2)) 

#     z_bp_df = bp_t_df.copy()
#     z_bp_df = z_bp_df[:30] #first 30 observations
#     z_bp_df = np.log(z_bp_df)

#     #dodgy code to break loop if standard deviation is 0
#     oops = 0
#     for column in z_bp_df.columns:
#         mean = z_bp_df[column].mean()
#         standard_deviation = np.std(z_bp_df[column], ddof = 1)
#         if type(standard_deviation) == pd.core.series.Series:
#             standard_deviation = standard_deviation.min()
#             z_bp_df[column] = z_bp_df[column].apply(lambda x: (x - mean) / standard_deviation)  
#         elif standard_deviation == 0: 
#             oops += 1
#         else:
#             z_bp_df[column] = z_bp_df[column].apply(lambda x: (x - mean) / standard_deviation)          
#     if oops == 1:
#         break

#      # calculating sum of squared spreads

#     ids = [column for column in z_bp_df.columns]

#     matrix = z_bp_df.iloc[0:0].copy()

#     matrix.insert(0, "horse", np.array(ids))
#     matrix = matrix.set_index("horse", drop = False)
#     del matrix["horse"]

#     for column in matrix.columns:
#         for row in matrix.index:
#             if column == row:
#                 matrix.loc[row, column] = np.nan
#             else: 
#                 matrix.loc[row, column] = ((z_bp_df[row] - z_bp_df[column]) ** 2).sum() or np.nan

#     for x in range(len(ids)):
#         for y in range(x, len(ids)):
#             matrix.iloc[x, y] = np.nan

#     horse_x = matrix.min(axis=1).idxmin()
#     horse_y = matrix.min().idxmin()
#     sss = matrix.min().min()

#     #regression setup
#     reg_df = bp_t_df[[horse_y, horse_x]][:30].copy() #non-standardised prices
#     reg_df = np.log(reg_df) 
#     reg_df['const'] = 1
#     reg_df['spread'] = reg_df[horse_y] - reg_df[horse_x]

#     #regression fit and results
#     reg = sm.OLS(endog=reg_df[horse_y], exog=reg_df[['const', horse_x]], missing='drop')

#     results = reg.fit()

#     constant = results.params[0]
#     beta = results.params[1]

#     spread = reg_df[horse_y].iloc[29].item() - constant - beta * reg_df[horse_x].iloc[29].item()

#     k = 50

#     if spread < 0:
#         #back to lay Y
#         bp_y = bp_t_df[horse_y].iloc[30]
#         lp_y = bp_t_df[horse_y].iloc[k]

#         win_side_y, loss_side_y = utils.payout(bp_y, 1, lp_y, '?')

#         #lay to back X
#         lp_x = bp_t_df[horse_x].iloc[30]
#         bp_x = bp_t_df[horse_x].iloc[k]

#         win_side_x, loss_side_x = utils.payout(bp_x, '?', lp_x, beta)

#     else:
#         #lay to back Y
#         lp_y = bp_t_df[horse_y].iloc[30]
#         bp_y = bp_t_df[horse_y].iloc[k]

#         win_side_y, loss_side_y = utils.payout(bp_y, '?', lp_y, 1)

#         #back to lay X
#         bp_x = bp_t_df[horse_x].iloc[30]
#         lp_x = bp_t_df[horse_x].iloc[k]

#         win_side_x, loss_side_x = utils.payout(bp_x, beta, lp_x, '?')
        
#     returns += win_side_y + win_side_x + loss_side_y + loss_side_x
    
# print(returns)

### Further Research 

__3.0__ **- [Herlemont (2004)](http://docs.finance.free.fr/DOCS/Yats/cointegration-en%5B1%5D.pdf) paper**

Herlemont describes in detail the econometrics of pairs trading for financial market assets. The following partly follows his commentary with some additional clarifications and discussion relating to horse racing.

**3.1 - Testing for mean reversion**

The aim is to identify odds that move together and whose spread is mean reverting. For the purposes of horse racing pairs, mean reversion is essential. Our objective is to capture prices whose spread has (temporarily) deviated from its mean. If this can be found, bets can be made to take advantage of the possible reversion.

A stochastic process $y_{t}$ that is weakly stationary has the following properties for all $t$:

* $E[y_{t}] = \mu < \infty$
* $var(y_{t}) = \gamma_{0} < \infty$
* $cov(y_{t}, y_{t-j}) = \gamma_{j} < \infty, j = 1, 2, 3 ...$

(constant mean, constant variance, covariance between two observations depends only on the distance in time between them)

A weakly stationary $I(0)$ series:
* Fluctuates around its mean with a finite variance that does not depend upon time.
* Is mean-reverting: it has tendency to return to its mean.
* Has limited memory; the effect of a shock dies out. Autocorrelations die out (fairly) rapidly.

With two horse's odds, $A_{t}$ and $B_{t}$, we look at $y_{t} = \log \frac{A_{t}}{B_{t}} = \log A_{t} - \log B_{t}$. This is once again the spread between the prices of the two horses, defined slightly differently. We want to find a pair which has a weakly stationary spread. We are interested in the ($AR(1)$) process 

$y_{t} = c + \theta y_{t-1} + \varepsilon_{t}$,

or the log odds ratio over time. If this is weakly stationary, it would suggest a mean reverting process. 

The three previous conditions, and a stability condition that $|\theta|<1$ (that the process $y_{t}$ is not a random walk or that it follows an eratic positive-to-negative pattern) must hold.
______

A Dickey-Fuller stationarity test can be carried out on the log ratio of the prices to test whether a process is weakly stationary. If we carry out the regression:

$\Delta y_{t} = \mu + \omega y_{t-1} + \varepsilon_{t}$

where the null hypothesis that $\omega = 0$ is that the 'true' relationship is $\Delta y_{t} = \mu + \varepsilon_{t} \Leftrightarrow y_{t} = \mu + y_{t-1} + \varepsilon_{t}$, or a random walk with starting point $y_{0} = \mu$.

If we can reject the null hypothesis, the price ratio is weakly stationary and thereby mean-reverting.

A Dickey-Fuller test is required for each possible pair of horses in a race, or $\frac{n(n-1)}{2}$ regressions, where $n$ is the number of horses.

While we are interested in the stochastic process $y_{t}$, we do not need to carry out the regression of $y_{t} = c + \theta y_{t-1} + \varepsilon_{t}$ for the purpose of finding pairs. This relationship between a pair of odds itself is not important to quantify. We are only interested in the features of the process. 
____

*In the previous analysis, the test for whether two odds formed a pair was to find the pair with the smallest sum of absolute differences over time in the standardised prices. That method would allow maximum 1 pair to be found per race, and the validity of that pair would not be confirmed statisticallyather. Rather, the pair's feasibilty for a trade would be tested for afterwards based on profitability. I have more confidence in the approach in this section.*

**3.2 - Screening pairs**

Herlemont describes rules to ensure that market neutrality is more achievable in pairs trading. The idea is to pick stocks with very similar characteristics like same industry and similar market betas, with the intention of minimising asymmetric shocks to the price of one stock and not the other. For example in the case of two stocks, the share on which you are long is a business heavily dependent on oil, while the other share is not, a surge in oil prices which dampens profitability of your long share will likely see its price fall, ruining the pairs trade. In the case of shares, the simplest solution would be to pick shares in similar industries with similar market betas (or with similar idiosyncratic risks).

For horses, the external factors influencing prices (news about runners, changing weather conditions, etc.) will usually always have asymmetric effects. This may be avoidable through picking horses with similar fundamental characteristics. However, this is very complicated. My hope is that the pair finding mechanism picks horses where this is already the case, because the market reacts the same way to news for these horse pairs.

We cannot follow a beta-based approach because there are not 'market-wide fluctuations' of the same sort. However, there is the fact that the implied probability of all horses in the market book is equal to approximately 1. Therefore, you could say that for a given change in implied probability for one horse, the sum of the changes in the odds of all the remaining horses is the negative the change for the given horse:

$\Delta O_{i} = - \sum_{j = 1, j \neq i}^{N_{h}} \Delta O_{j} $

There is therefore interdependence between all prices across the market. It's possible that this will cause an endogeneity problem in regressions between separate horses, as the changes in the dependent variable necessarily impact the explanatory variable. However, the impact is likely to be very small, and will be smaller the greater the number of horses. 

*In Bebbington's analysis, he describes that betting £1 on one of the horses and £$\beta$ on the other creates a market neutral bet. This is incorrect, and it appears that he has misunderstood hedging in this context. In that analysis, $\beta = \frac{y_{t}}{x_{t}}$, and therefore he is simply considering the ratio of the prices of the horses, the same ratio considered when determining the optimal stake for two given prices in a hedge. It is correct that on a single horse this creates a market neutral bet, however neutrality in horse racing means neutral to the outcome of the race. Any bet neutral to the race outcome is definitively neutral to the market. When betting on separate horses, the bets on each horse must be made neutral separately. Additionally, the use of $\beta$ in staking is unneccesary. Consider the case where £$BS$ has been bet on horse A at price $BP$. Now, horse A is priced at $LP$. The optimal stake to bet on LP is £$LS = \frac{BS * BP}{LP}$. In the aforementioned regression, $BS = 1$, hence $\beta = \frac{y_{t} * 1}{x_{t}}$ is the optimal stake only for bets of £1, otherwise it would be $S*\beta$. More importantly, using the estimated $\beta$ to find the an approximation of the optimal stake makes no sense when you can simply find the optimal stake with the aforementioned equation.*

**3.3 - Trading rules**

Timing rules must be added. 

Herlemont's basic rule is "to open a position when the ratio of two share prices hits the 2 rolling standard deviation [difference from the 130-day rolling mean] and close it when the ratio returns to the mean."

To avoid opening a position on stocks that are deviating from the mean and are going to deviate further, Herlemont describes that "the position is not opened when the ratio breaks the two-standard-deviations limit for the first time, but rather when it crosses it to revert to the mean again."

This can be achieved with the horse odds, of course in far smaller time scales. The current dataset is in 5-minute intervals for the three hours before a race; this should likely be expanded.

Stop losses should be included and trade length should also be limited.

Rules:
1. Trade on pairs whose spread is reapproaching the mean from a deviated position
2. Stop loss at x% of the initial position
3. Don't hold open pairs trades for longer than x hours. 

It should be possible to quantify the average length of time required for a mean reversion and therefore the maximum logical time to hold open a position by looking at past data.

**3.4 - Other tests and considerations**

1. It should be ensured that the regression results of one price on another are not spurious (as with the regression in 2.5). $\beta$ could be statistically meaningless if it is, meaning that it makes no sense to use it.
2. I will also test whether $y_{t} = c + \theta y_{t-1} + \varepsilon_{t}$ is $I(1)$, or difference stationary. If we can rule this out, this gives more confidence in the 'weak-stationarity' of the spread over time.
3. I will look out for $\omega$ in the DF test that are close to 1 yet pass the DF test. They will have lots of features of a random walk, so the pairs exercise might be meaningless.
4. Structural breaks (in this case, large instantaneous jumps in the spread) may make series that are stationary on either side of the break appear non-stationary. This is hard to account for in testing. 

In [47]:
#3.1 - testing for mean reversion

#using non-standardised log price data
back_prices = np.log(bp_t_df[:30]).copy()
back_prices.head()

,h10460778,h12464466,h13168800,h15609001,h15888970,h21809948,h22972495,h26684163,h3518738,h5826525,h6406084,h6670738
0,3.828641,2.397895,3.332205,2.232163,1.458615,1.481605,1.791759,3.401197,3.583519,4.60517,2.944439,4.007333
1,3.828641,2.394252,3.338967,2.232163,1.465568,1.481605,1.791759,3.401197,3.630190,4.60517,2.969388,4.007333
2,3.828641,2.351375,3.367296,2.219203,1.456287,1.481605,1.791759,3.409827,3.588506,4.60517,2.970414,4.007333
3,3.828641,2.351375,3.396185,2.219203,1.458615,1.481605,1.791759,3.401197,3.637586,4.60517,2.979095,4.007333
4,3.828641,2.370244,3.401197,2.219203,1.456287,1.481605,1.791759,3.401197,3.654288,4.60517,2.983153,4.007333


In [48]:
#create dataframe where each column is log(horse a's prices) - log(horse b's prices). one new column for all n(n-1)/2 possible pairs

#use itertools to find all possible comination pairs
combos = list(itertools.combinations(back_prices.columns, 2))

#creating dataframe
for pair in combos:
    if pair == combos[0]:
        new_series = back_prices[pair[0]] - back_prices[pair[1]]
        dickey_fuller_df = pd.DataFrame(new_series)
    else:
        new_series = back_prices[pair[0]] - back_prices[pair[1]]
        dickey_fuller_df = pd.concat([dickey_fuller_df, new_series], axis=1)
        
#naming columns
dickey_fuller_df.columns = [pair[0] + "_" + pair[1] for pair in combos]

dickey_fuller_df['const'] = 1

dickey_fuller_df.head()

,h10460778_h12464466,h10460778_h13168800,h10460778_h15609001,h10460778_h15888970,h10460778_h21809948,h10460778_h22972495,h10460778_h26684163,h10460778_h3518738,h10460778_h5826525,h10460778_h6406084,...,h26684163_h5826525,h26684163_h6406084,h26684163_h6670738,h3518738_h5826525,h3518738_h6406084,h3518738_h6670738,h5826525_h6406084,h5826525_h6670738,h6406084_h6670738,const
0,1.430746,0.496437,1.596479,2.370026,2.347037,2.036882,0.427444,0.245122,-0.776529,0.884202,...,-1.203973,0.456758,-0.606136,-1.021651,0.639080,-0.423814,1.660731,0.597837,-1.062894,1
1,1.434389,0.489674,1.596479,2.363074,2.347037,2.036882,0.427444,0.198451,-0.776529,0.859253,...,-1.203973,0.431809,-0.606136,-0.974980,0.660802,-0.377143,1.635782,0.597837,-1.037945,1
2,1.477266,0.461346,1.609438,2.372355,2.347037,2.036882,0.418815,0.240135,-0.776529,0.858227,...,-1.195343,0.439412,-0.597506,-1.016664,0.618092,-0.418827,1.634756,0.597837,-1.036919,1
3,1.477266,0.432457,1.609438,2.370026,2.347037,2.036882,0.427444,0.191055,-0.776529,0.849547,...,-1.203973,0.422103,-0.606136,-0.967584,0.658492,-0.369747,1.626076,0.597837,-1.028239,1
4,1.458398,0.427444,1.609438,2.372355,2.347037,2.036882,0.427444,0.174353,-0.776529,0.845488,...,-1.203973,0.418044,-0.606136,-0.950882,0.671135,-0.353045,1.622017,0.597837,-1.024180,1


In [56]:
#dickey fuller test on each column

#regression fit and results in horizontal dataframe format (1 column per pair)
# dickey_fuller_results = {}
# for column in dickey_fuller_df:
#     reg = sm.OLS(endog = dickey_fuller_df[column].diff(), exog = dickey_fuller_df[['const', column]], missing = 'drop')
#     results = reg.fit()
#     dickey_fuller_results[column] = results.tvalues[1]

#regression fit and results in vertical dataframe format (column for pair id, column for dickey fuller test result)
dickey_fuller_results = {'pair' : [], 'coef' : [], 'critical_value' : []}

for column in dickey_fuller_df:
    if column == 'const':
        break
    reg = sm.OLS(endog = dickey_fuller_df[column].diff(), exog = dickey_fuller_df[['const', column]].shift(1), missing = 'drop')
    results = reg.fit()
    dickey_fuller_results['pair'].append(column)
    dickey_fuller_results['coef'].append(results.params[1])
    dickey_fuller_results['critical_value'].append(results.tvalues[1])

dickey_fuller_results_df = pd.DataFrame(dickey_fuller_results)

dickey_fuller_results_df

#compare to -3.58 from the MacKinnon tables for 1% significance level, -2.93 for 5% significance level

,pair,coef,critical_value
0,h10460778_h12464466,-0.130830,-1.507704
1,h10460778_h13168800,-0.130794,-1.408255
2,h10460778_h15609001,-0.091746,-1.392023
3,h10460778_h15888970,-0.080285,-1.289583
4,h10460778_h21809948,-0.094062,-1.397878
...,...,...,...
61,h3518738_h6406084,0.013514,0.139708
62,h3518738_h6670738,-0.260773,-2.085108
63,h5826525_h6406084,0.547812,3.355950
64,h5826525_h6670738,-0.459423,-2.811947


In [59]:
pair_index = dickey_fuller_results_df['critical_value'].idxmin()
pair_cv = dickey_fuller_results_df['critical_value'].min()
pair_ids = dickey_fuller_results_df['pair'].loc[pair_index]
pair_coef = dickey_fuller_results_df['coef'].loc[pair_index]

horse_a = pair_ids.split("_", 1)[0]
horse_b = pair_ids.split("_", 1)[1]
pair_df = np.log(bp_t_df[[horse_a, horse_b]])
pair_df['spread'] = pair_df[horse_a] - pair_df[horse_b]

pair_spread_sd = np.std(pair_df['spread'], ddof = 1)
pair_spread_mean = pair_df['spread'].mean()

pair_df['deviation_2sd'] = pair_df['spread'].where(abs(pair_df['spread']) - abs(pair_spread_mean) > 2 * pair_spread_sd)
pair_df['deviation_1sd'] = pair_df['spread'].where(abs(pair_df['spread']) - abs(pair_spread_mean) > pair_spread_sd)

print("Pair row index: " + str(pair_index), ", Pair ids: " + str(pair_ids), ", Pair DF test critical value: " + str(pair_cv), ", Pair theta: " + str(pair_coef))
print("Pair average spread: " + str(pair_spread_mean), ", Pair spread standard deviation: " + str(pair_spread_sd))

if pair_cv < - 3.58:
    print(f"{horse_a} and {horse_b} form a pair. Stationarity is not rejected at the 1% signficance level.")
elif pair_cv < - 2.93:
    print(f"{horse_a} and {horse_b} form a pair. Stationarity is not rejected at the 5% signficance level.")
else:
    print("No pair found")

Pair row index: 11 , Pair ids: h12464466_h13168800 , Pair DF test critical value: -3.6832158526031074 , Pair theta: -0.560536560646371
Pair average spread: -0.9722521739972682 , Pair spread standard deviation: 0.08116380916425366
h12464466 and h13168800 form a pair. Stationarity is not rejected at the 1% signficance level.


In [60]:
pair_df

,h12464466,h13168800,spread,deviation_2sd,deviation_1sd
0,2.397895,3.332205,-0.934309,NaN,NaN
1,2.394252,3.338967,-0.944715,NaN,NaN
2,2.351375,3.367296,-1.015921,NaN,NaN
3,2.351375,3.396185,-1.044810,NaN,NaN
4,2.370244,3.401197,-1.030954,NaN,NaN
5,2.351375,3.401197,-1.049822,NaN,NaN
6,2.351375,3.401197,-1.049822,NaN,NaN
7,2.371178,3.409827,-1.038649,NaN,NaN
8,2.397895,3.409827,-1.011931,NaN,NaN
9,2.397895,3.465736,-1.067841,NaN,-1.067841
